In [29]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import itertools as it
from itertools import product
from collections import Counter, defaultdict, deque
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
import copy
sns.set_style('white')
sns.set_context('notebook', font_scale=1.3)

from agents import Agent
from evaluation import get_util
from joblib import Parallel, delayed
# from model_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# import ipyparallel as ipp 
# rc = ipp.Client(profile='default', cluster_id='')
# ipp.register_joblib_backend()

# Parsing

In [31]:
from mouselab import MouselabEnv
from distributions import Categorical, Normal

def make_envs(cost=1.00, n=100, seed=None,variance_structure="constant_high",branching=[4,1,2]):
    if seed is not None:
        np.random.seed(seed)
    
    depth = len(branching)
    
    if variance_structure is "constant_high":
        sigmas = np.concatenate( (np.array([0]),20*np.ones(depth)))
    if variance_structure is "increasing":
        sigmas = [0, 2, 4, 20]
    if variance_structure is "decreasing":
        sigmas = [0,20,4,2]
    if variance_structure is "constant_low":
        sigmas = np.concatenate( (np.array([0]),3*np.ones(depth)))
        
    def reward(depth):
        if depth > 0:
            return Normal(0, sigmas[depth]).to_discrete(6)
        return 0.

    envs = [MouselabEnv.new_symmetric(branching, reward)
            for _ in range(n)]
    for env in envs:
        env.cost=-cost
    
    return envs

In [32]:
cost = 1
envs = make_envs(cost,10,None,"increasing",branching=[2,2])
env = envs[0]

In [33]:
env.paths

[[1, 2], [1, 3], [4, 5], [4, 6]]

In [34]:
env._state

(0, Cat, Cat, Cat, Cat, Cat, Cat)

In [35]:
env.tree

[[1, 4], [2, 3], [], [], [5, 6], [], []]

In [36]:
# fancy initialization function, but doesn't seem necessary
# simpler version built into the class
def build_path(r):   
    if env.tree[r] == []:
        return ([[]], [0])
    paths = []
    path_moves = []
    for n in env.tree[r]:
        new_paths, new_path_moves = build_path(n)
        for i in range(len(new_paths)):
            new_paths[i].insert(0,n)
            new_move = int(hasattr(env._state[n],'sample'))
            path_moves.append(new_path_moves[i]+new_move)
            paths.append(new_paths[i])
    return (paths,path_moves)

In [37]:
def option_util(x,sigma):
    return sigma*scipy.stats.norm.pdf(x/sigma) - np.abs(x)*scipy.stats.norm.cdf(-np.abs(x)/sigma)

In [38]:
def get_all_options(env):
    paths = env.paths #list of all paths
    avail_moves = [0,]*len(paths) #list of moves available in each path
    path_obs = [] #value of observed nodes in each path
    path_nodes = [] #the unobserved nodes of each path
    path_stds = [] #the std deviation of the unobserved nodes of each path
    
    options = [] #list of all options
    option_utils = [] #list of the utility of each option
    
    for i in range(len(paths)):
        stds = []
        nodes = []
        obs = 0
        
        for node in paths[i]:
            if hasattr(env._state[node],'sample'):
                stds.append(env._state[node].var())
                nodes.append(node)
                avail_moves[i] += 1
            else:
                obs += env._state[node]
                
        path_obs.append(obs)
        path_stds.append(stds)
        path_nodes.append(nodes)
        
        for j in range(avail_moves[i]):
            options.append((i,j+1))
    max_obs = np.max(path_obs)
    
    for option in options:
        path, obs = option
        option_utils.append(option_util(path_obs[path]-max_obs,np.sqrt(np.sum(np.sort(path_stds[path])[::-1][:obs]))) + obs*env.cost)
    
    return options, option_utils, path_nodes, path_stds, path_obs, avail_moves

In [39]:
def pick_option_moves(env):
    options, option_utils, path_nodes, path_stds, path_obs, avail_moves = get_all_options(env)
    
    #c is for chosen
    cpath, cobs = options[np.random.choice(np.arange(len(options))[option_utils == np.max(option_utils)])]
    cpath_stds = np.array(path_stds[cpath]) 
    b = np.random.random(cpath_stds.size)
    cpath_nodes = np.array(path_nodes[cpath])
    return cpath_nodes[np.lexsort((b,cpath_stds))[::-1][:cobs]]

In [40]:
env.paths[1]

[1, 3]

In [41]:
import time
t = time.process_time()
for i in range(1000):
#     print(get_options(env))
    pick_option_moves(env)
elapsed =time.process_time() - t 
print(elapsed)

1.2885142919999995


In [42]:
def all_option_insts(path_nodes,path_stds,n_obs):
    insts = [[]]
    n_remaining_obs = n_obs

    vals, inverse, count = np.unique(path_stds, return_inverse=True,
                                  return_counts=True)
    rows, cols = np.where(inverse == np.arange(len(vals))[:, np.newaxis])
    _, inverse_rows = np.unique(rows, return_index=True)
    res = np.split(cols, inverse_rows[1:])

    for i in range(len(res)):
        new_insts = []

        n_new_nodes = len(res[-i-1])
        if n_new_nodes < n_remaining_obs:
            n_remaining_obs -= n_new_nodes
        else:
            n_new_nodes = n_remaining_obs
            n_remaining_obs = 0  

        for new_nodes in it.permutations(res[-i-1],n_new_nodes):
            for inst in insts:
                new_insts.append(inst + list(np.array(path_nodes)[list(new_nodes)]))
        insts = new_insts
        if n_remaining_obs == 0:
            break
            
    return insts

In [43]:
import time
path_stds= [10.00,10.00,9,8.0,8,7,7,6]
path_nodes = np.arange(len(path_stds))
t = time.process_time()
for i in range(1000):
#     print(get_options(env))
    all_option_insts(path_nodes,path_stds,6)
elapsed =time.process_time() - t 
print(elapsed)

0.10718614299999984


In [44]:
path_stds= [10.00,10.00,9,8.0,8,7,7,6]
path_nodes = np.arange(len(path_stds))
all_option_insts(path_nodes,path_stds,6)

[[0, 1, 2, 3, 4, 5],
 [1, 0, 2, 3, 4, 5],
 [0, 1, 2, 4, 3, 5],
 [1, 0, 2, 4, 3, 5],
 [0, 1, 2, 3, 4, 6],
 [1, 0, 2, 3, 4, 6],
 [0, 1, 2, 4, 3, 6],
 [1, 0, 2, 4, 3, 6]]

The following equation gives the likelihood of a click sequence in this model:

$$ l(C) = \sum_{O \in \mathcal{O}(C)} l(O)\mathbb{P}(C|O)$$

where $C$ is a click sequence, and $\mathcal{O}(C)$ is the set of option sequences that $C$ could parse into. To get the likelihood of an option sequence, we use the following equation:

$$ l(O) = \prod_{o \in O} \mathbb{P}_{DC}(o) $$

$$ \mathbb{P}(C|O) = \prod_{o \in O} \mathbb{P}(i_o|o, s) $$ 

where $\mathbb{P}_{DC}(o)$ is the probability of a given option in our model. We assume a generative model that picks a random move with probability $p_r$, or otherwise picks from one of the available options under the directed cognition model. This gives us the following equation: 

$$\mathbb{P}_{DC}(o) = (1-p_r)*s(o)+p_r*\alpha$$ 

$s(o)$ is the softmax probability of our option among all available directed cognition options, which, for a given temperature parameter $t = \frac{1}{\beta}$ is equal to:

$$ s(o)= \frac{e^{\beta v(o)}}{\sum_{o'} e^{\beta v(o')}} $$

where $v(o)$ is the value of an option using the directed cognition model. We define $\alpha$ as the probability that an option in the sequence would have been generated by the error process, and it is defined as:

$$\alpha = \prod_{k = 1}^ \text{length(o)} \frac{1}{n_{ac}-k+1}$$

where $n_{ac}$ is the number of available clicks.

In [45]:
alphas = np.array([[np.prod([1/(nac-k+1) for k in range(1,obs+1)]) for obs in range(1,min(len(env.paths[0]),nac)+1)] for nac in range(1,13)])

In [46]:
alphas[9][0]

0.10000000000000001

In [47]:
np.prod([1/(10-k+1) for k in range(1,1+1)])

0.10000000000000001

In [100]:
def parse_options(env,click_sequence,t=1,p_rand=0.0001):
    if click_sequence == []:
        return True, [[]], [1]
    option_insts = dict() #list of all possible option instantiations
    option_seqs = []
    likelihoods = []
    done = False
    
    paths = env.paths
    options, option_utils, path_nodes, path_stds, path_obs, avail_moves = get_all_options(env)

    for option in options:
        path, obs = option
        option_insts[option] = all_option_insts(path_nodes[path],path_stds[path],obs)
        
    #single click options
    sc_opt = (-1,1)
    options.append(sc_opt)
    option_utils.append(-np.inf)
    option_insts[sc_opt] = [[a] for a in env.actions(env._state)]
    n_available_clicks = len(option_insts[sc_opt])
    
    #end click options
    end_opt = (-99,1)
    options.append(end_opt)
    option_utils.append(0)

    option_insts[end_opt] = [[env.term_action]]
            
    option_utils = np.array(option_utils)
#     print(option_utils)
    for i in range(1,min(len(paths[0]),len(click_sequence))+1):  
        for j in range(len(options)):
            option = options[j]
            n_insts = len(option_insts[option])
            for inst in option_insts[option]:      
                if np.array_equal(click_sequence[:i],inst):
                    copy_env = copy.deepcopy(env)                   
                    for a in click_sequence[:i]:
                        copy_env._step(a)
                        
                    will_done, remaining, rem_likelihoods = parse_options(copy_env,click_sequence[i:],t,p_rand)
                    done = done or will_done
                    
                    if done:
                        for k in range(len(remaining)): 
                            latter = remaining[k]
                            option_seqs.append([option]+latter) 
#                             print('doing'+str(option)+str(latter))
#                             print(option_utils[j])
#                             print(np.exp(1/t*option_utils))
                            l_opt_seq = (1-p_rand)*np.exp(1/t*option_utils[j])/np.sum(np.exp(1/t*option_utils))
#                             print(l_opt_seq)
                            l_opt_seq += p_rand*np.prod([1/(n_available_clicks-k+1) for k in range(1,option[1]+1)])
#                             print(l_opt_seq)
                            l_opt_seq *= rem_likelihoods[k]
#                             print(rem_likelihoods[k])
                            likelihoods.append(l_opt_seq*1/n_insts)
#                             print(n_insts)
    
    return done, option_seqs, likelihoods

In [101]:
participants[2][6]

(<mouselab.MouselabEnv at 0x7f6862085240>, [1, 9, 5, 6, 2, 10, 8, 7, 3, 4])

In [102]:
env,trial = copy.deepcopy(participants[2][6])
trial.append(env.term_action)
print(trial)
a,b,c = parse_options(env,trial,t=2,p_rand=0.01)
print(np.log(sum(c)))
a,b,c

[1, 9, 5, 6, 2, 10, 8, 7, 3, 4, 13]
-22.4594733701


(True,
 [[(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (0, 1),
   (1, 1),
   (-1, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (0, 1),
   (1, 1),
   (-99, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (0, 1),
   (-1, 1),
   (-1, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (0, 1),
   (-1, 1),
   (-99, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (-1, 1),
   (1, 1),
   (-1, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (-1, 1),
   (1, 1),
   (-99, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (-1, 1),
   (-1, 1),
   (-1, 1)],
  [(0, 1),
   (4, 1),
   (2, 1),
   (2, 1),
   (0, 1),
   (4, 1),
   (3, 1),
   (2, 1),
   (-1, 1),
   (-1, 1),
   (-99, 1)

In [103]:
print(np.log(sum(c)))

-22.4594733701


In [91]:
env.ground_truth

array([  0,  -5,  10,  -5,  10,   5,   5, -10,   5,  10,  -5,  10,  10])

In [92]:
env = envs[2]
trial = [6]
trial.append(env.term_action)
print(trial)
a,b,c = parse_options(env,trial,t=2,p_rand=0.01)
print(np.log(sum(c)))
a,b,c

[6, 7]
[ 0.582 -0.231  0.582 -0.231  0.582 -0.231  0.582 -0.231   -inf  0.   ]
[ 0.582 -0.231  0.582 -0.231  0.582 -0.231 -0.995   -inf  0.   ]
doing(-1, 1)[]
-inf
[ 1.338  0.891  1.338  0.891  1.338  0.891  0.608  0.     1.   ]
0.0
0.00166666666667
1
6
doing(-99, 1)[]
0.0
[ 1.338  0.891  1.338  0.891  1.338  0.891  0.608  0.     1.   ]
0.119362552352
0.121029219019
1
1
doing(3, 1)[(-1, 1)]
0.582084325591
[ 1.338  0.891  1.338  0.891  1.338  0.891  1.338  0.891  0.     1.   ]
0.133584383999
0.135012955428
0.000277777777778
1
doing(3, 1)[(-99, 1)]
0.582084325591
[ 1.338  0.891  1.338  0.891  1.338  0.891  1.338  0.891  0.     1.   ]
0.133584383999
0.135012955428
0.121029219019
1
[ 0.582 -0.231  0.582 -0.231  0.582 -0.231 -0.995   -inf  0.   ]
doing(-1, 1)[]
-inf
[ 1.338  0.891  1.338  0.891  1.338  0.891  0.608  0.     1.   ]
0.0
0.00166666666667
1
6
doing(-99, 1)[]
0.0
[ 1.338  0.891  1.338  0.891  1.338  0.891  0.608  0.     1.   ]
0.119362552352
0.121029219019
1
1
doing(-1, 1)[(-1, 1

(True,
 [[(3, 1), (-1, 1)],
  [(3, 1), (-99, 1)],
  [(-1, 1), (-1, 1)],
  [(-1, 1), (-99, 1)]],
 [3.7503598729920606e-05,
  0.016340512552852016,
  5.6689342403628116e-08,
  2.4699840616105385e-05])

In [82]:
print(np.log(sum(c)))

-4.11907278407


In [64]:
env,trial = participants[2][6]

In [68]:
trial

[1, 9, 5, 6, 2, 10, 8, 7, 3, 4]

In [66]:
a,b,c = parse_options(env,trial,t=2,p_rand=0.01)

In [67]:
print(np.log(sum(c)))

-21.6644755042


# Modeling

In [54]:
def make_env(mu=0, sigma=5, quantization=4, cost=1.00, seed=None, branching=[3,1,2], **kwargs):
    if seed is not None:
        np.random.seed(seed)

    def reward(depth):
        if depth > 0:
            x = np.array([-2,-1,1,2])
            return Categorical(mu + sigma * x)
        return 0.

    return MouselabEnv.new_symmetric(branching, reward, cost=cost, **kwargs)

env = make_env(ground_truth=False)

In [55]:
from analysis_utils import *
VERSION = 'c1.1'
exp_data = get_data(VERSION, '../experiment/data')

pdf = exp_data['participants']
pdf = pdf.loc[pdf.completed].copy()
print(f'{len(pdf)} participants')
complete = list(pdf.index)

def extract(q):
    return list(map(int, q['click']['state']['target']))

mdf = exp_data['mouselab-mdp'].query('pid == @complete').copy()
mdf['clicks'] = mdf.queries.apply(extract)
mdf['n_clicks'] = mdf.clicks.apply(len)
mdf['thinking'] = mdf['rt'].apply(get(0, default=0))

tdf = mdf.query('block == "test"').copy()
tdf.trial_index -= tdf.trial_index.min()
tdf.trial_index = tdf.trial_index.astype(int)
tdf.trial_id = tdf.trial_id.astype(int)

# pdf['total_time'] = exp_data['survey'].time_elapsed / 60000

pdf['n_clicks'] = tdf.groupby('pid').n_clicks.mean()
pdf['score'] = tdf.groupby('pid').score.mean()
pdf['thinking'] = mdf.groupby('pid').thinking.mean()

Error importing rpy2
60 participants


In [56]:
import json
def excluded_pids():
    sdf = exp_data['survey-multi-choice'].query('pid == @complete').copy()
    sdf = pd.DataFrame(list(sdf.responses), index=sdf.index)
    correct = pd.Series(['-$10 to $10', '$1', '1 cent for every $1 you make in the game'])
    fail_quiz = (sdf != correct).sum(axis=1) > 1
    no_click = mdf.query('block == "train_inspector"').groupby('pid').n_clicks.sum() == 0
    return fail_quiz | no_click

exclude = excluded_pids()
tdf['exclude'] = list(exclude.loc[tdf.pid])
tdf = tdf.query('~exclude').copy().drop('exclude', axis=1)
print(f'excluding {exclude.sum()} out of {len(exclude)} partipicants')

excluding 9 out of 60 partipicants


In [62]:
def get_env(state_rewards):
    state_rewards[0] = 0
    return make_env(ground_truth=state_rewards)
tdf['env'] = tdf.state_rewards.apply(get_env)

In [63]:
j = 0
participants = dict()
for i, row in tdf.iterrows():
    if row['pid'] in participants.keys():
        participants[row['pid']].append((row['env'],row['clicks']))
    else:
        participants[row['pid']] =[(row['env'],row['clicks'])]

In [ ]:
keys = list(participants.keys())

In [58]:
clicks = [np.sum([len(trial) for env,trial in participants[p]]) for p in keys]

NameError: name 'keys' is not defined

In [59]:
np.argmax(clicks)

NameError: name 'clicks' is not defined

In [32]:
keys[1]

2

In [39]:
clicks[11]

182

In [30]:
np.sum([len(trial) for env,trial in participants[17]])

241

In [166]:
p_errs = np.linspace(0.01,0.25, 25)
temp = np.logspace(-5,1, 50)

In [104]:
def dc_log_likelihood(env, trial, temp, p_error):
    env = copy.deepcopy(env)
    trial = copy.deepcopy(trial)
    trial.append(env.term_action)
    done, option_seqs, likelihoods = parse_options(env,trial,t=temp,p_rand=p_error)
    return np.log(np.sum(likelihoods))

In [105]:
p = list(participants.keys())[0]
env,trial = participants[p][6]
dc_log_likelihood(env, trial, 1, 0.25)

-9.6839091543431586

In [194]:
import time
t = time.process_time()
for i in range(1000):
    dc_log_likelihood(env, trial, 1, 0.01)
elapsed =time.process_time() - t 
print(elapsed)

156.80737954


In [ ]:
temp, p_error = 1,.01
p = 2
data = participants[p]
t = time.process_time()
print(sum(dc_log_likelihood(env, trial, temp, p_error) for env, trial in data))
elapsed =time.process_time() - t 
print(elapsed)

In [195]:
def mle(data):
# You can adjust the temperature bounds if you think the MLE
# is not in the bounds below. Don't change the p_error bounds.
    bounds = [
        (1e-5, 1e2),  # temp
        (0, .25)  # p_error
    ]
    def loss(x):
        temp, p_error = x
        return -sum(dc_log_likelihood(env, trial, temp, p_error)
                    for env, trial in data)
    res = scipy.optimize.minimize(loss, (1,.01),bounds = bounds)
    temp, p_error = res.x
    return {'temp': res.x[0], 'p_error': res.x[1], 'logp': -res.fun}

In [196]:
def mle_wrap(p):
    d = mle(participants[p])
    d['participant']:p
    return d

In [237]:
mles=dict()
p = list(participants.keys())[0]
t = time.process_time()
mles[p] = mle_wrap(p)
elapsed =time.process_time() - t 
print(elapsed)
print(mles[p])

144.59019641499998
{'temp': 100.0, 'p_error': 0.25, 'logp': -254.29114630683654}


In [89]:
sum(dc_log_likelihood(env, trial, 100.0, 0.25) for env, trial in data)

-3472.9150719069462

In [90]:
# mles = dict()
# for p in participants.keys():
#     print(p)
#     data = participants[p]
#     mles[p] = mle(data)
mles = Parallel(n_jobs=20)(delayed(mle_wrap)(p)
                           for p in participants.keys())

In [93]:
np.save('dc_mles',mles)

In [95]:
np.load('dc_mles.npy')

array([{'temp': 100.0, 'p_error': 0.25, 'logp': -3472.9150719069462}], dtype=object)

# Scratch

In [21]:
def parse_options_combined(env,click_sequence,temps=np.logspace(-3,1,50),p_errs=np.linspace(0,0.25,25)):
    
    #count things
    n_temps = len(temps)
    n_p_errs = len(p_errs)
    
    #base case
    if click_sequence == []:
        return True, [[]], [np.ones((n_temps,n_p_errs))]
    
    #get the info you'll need for parsing
    paths = env.paths
    options, option_utils, path_nodes, path_stds, path_obs, avail_moves = get_all_options(env)
    
    option_insts = dict() #list of all possible option instantiations
    for option in options:
        path, obs = option
        option_insts[option] = all_option_insts(path_nodes[path],path_stds[path],obs)
    
    #set up your return values
    option_seqs = []
    likelihoods = []
    done = False
        
    #single click options
    sc_opt = (-1,1)
    options.append(sc_opt)
    option_utils.append(-np.inf)
    option_utils = np.array(option_utils)
    option_insts[sc_opt] = [[a] for a in env.actions(env._state)]
    n_available_clicks = len(option_insts[sc_opt])-1
    
    #parsing
    for i,j in product(range(1,min(len(paths[0]),len(click_sequence))+1),range(len(options))):  
            option = options[j]
            for inst in option_insts[option]:      
                if np.array_equal(click_sequence[:i],inst):
                    copy_env = copy.deepcopy(env)                   
                    for a in click_sequence[:i]:
                        copy_env._step(a)
                        
                    will_done, remaining, rem_likelihoods = parse_options_combined(copy_env,click_sequence[i:])
                    done = done or will_done
                    
                    if done:
                        for k in range(len(remaining)): 
                            latter = remaining[k]
                            option_seqs.append([option]+latter)
                            
                            for l,m in product(range(n_temps),range(n_p_errs)):
                                l_opt_seq = np.zeros((n_temps,n_p_errs))
                                t = temps[l]
                                p_err = p_errs[m]
                                l_opt_seq[l,m] = (1-p_err)*np.exp(1/t*option_utils[j])/np.sum(np.exp(1/t*option_utils[:-1]))
                                l_opt_seq[l,m] += p_err*np.prod([1/(n_available_clicks-k+1) for k in range(1,option[1]+1)])
                                l_opt_seq *= rem_likelihoods[k][l,m]
                            
                                likelihoods.append(l_opt_seq)
    return done, option_seqs, likelihoods